# 🎄 12 Days of Demos: North Pole Modernization Office (NPMO)

## 🧠 Using Unity Catalog Functions for AI Agents via Managed MCP

To empower our AI agents (and elves) to answer complex questions about North Pole operations, we are creating a suite of **SQL Functions** in Unity Catalog. These functions encapsulate business logic, making it easy to retrieve metrics on everything from *Reindeer Health* to *Toy Production Quality*. We'll show how we can then use **Databricks Managed MCP Servers** to provide these tools directly to an agent!

*Let's give our agents the tools they need to save Christmas!* 🎁


To empower our AI agents (and elves) to answer complex questions about North Pole operations, we are creating a suite of **SQL Functions** in Unity Catalog. These functions encapsulate business logic, making it easy to retrieve metrics on everything from *Reindeer Health* to *Toy Production Quality*. We'll show how we can then use Databricks Managed MCP Servers to provide these tools directly to an agent!

*Let's give our agents the tools they need to save Christmas!* 🎁

In [ ]:
# Configuration
# TODO: Update these values for your environment
catalog_name = "12daysofdemos"
schema_name = "npmo"

### 🏭 Workshop Production Metrics
*Tracking material usage to ensure we don't run out of glitter!* ✨

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_workshop_material_totals')()
RETURNS TABLE(total_plastic_kg DOUBLE, total_wood_kg DOUBLE, total_metal_kg DOUBLE, total_fabric_kg DOUBLE)
COMMENT 'Returns total usage of all materials in the workshop.'
RETURN (
  SELECT 
    SUM(materials_plastic_kg) as total_plastic_kg,
    SUM(materials_wood_kg) as total_wood_kg,
    SUM(materials_metal_kg) as total_metal_kg,
    SUM(materials_fabric_kg) as total_fabric_kg
  FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production')
);
""")

### 🧝 Elf Team Performance
*Monitoring quality scores to find out which team deserves extra hot cocoa.* ☕

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_team_quality_metrics')(team_id STRING)
RETURNS TABLE(team_id STRING, avg_paint_quality DOUBLE, avg_assembly_score DOUBLE, total_defects BIGINT)
COMMENT 'Returns quality metrics for a specific elf team.'
RETURN (
  SELECT 
    elf_team_id as team_id,
    AVG(paint_quality) as avg_paint_quality,
    AVG(assembly_score) as avg_assembly_score,
    SUM(defect_count) as total_defects
  FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production')
  WHERE elf_team_id = team_id
  GROUP BY elf_team_id
);
""")

### 📍 Workshop Location Stats
*Comparing efficiency across our North, South, East, and West workshops.* 🧭

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_workshop_location_metrics')(location STRING)
RETURNS TABLE(workshop_location STRING, avg_production_time DOUBLE, avg_carbon_footprint DOUBLE)
COMMENT 'Returns performance metrics for a specific workshop location.'
RETURN (
  SELECT 
    workshop_location,
    AVG(production_time_minutes) as avg_production_time,
    AVG(carbon_footprint_kg) as avg_carbon_footprint
  FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production')
  WHERE workshop_location = location
  GROUP BY workshop_location
);
""")

### 🧸 Toy Defect Rates
*Ensuring every toy brings joy, not disappointment.* 🔧

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_toy_defect_rate')(category STRING)
RETURNS TABLE(toy_category STRING, defect_rate DOUBLE, total_produced BIGINT)
COMMENT 'Returns the percentage of toys with defects for a given category.'
RETURN (
  SELECT 
    toy_category,
    CAST(SUM(CASE WHEN defect_count > 0 THEN 1 ELSE 0 END) AS DOUBLE) / COUNT(*) as defect_rate,
    COUNT(*) as total_produced
  FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production')
  WHERE toy_category = category
  GROUP BY toy_category
);
""")

### ⏰ Shift Performance
*Analyzing productivity during the Morning, Afternoon, and Night shifts.* 🌙

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_shift_performance')(shift_name STRING)
RETURNS TABLE(shift STRING, avg_quality_tier STRING, avg_production_time DOUBLE)
COMMENT 'Returns average production time and most common quality tier for a shift.'
RETURN (
  SELECT 
    shift,
    mode(quality_tier) as avg_quality_tier,
    AVG(production_time_minutes) as avg_production_time
  FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production')
  WHERE shift = shift_name
  GROUP BY shift
);
""")

### 📬 Delivery Preferences
*Chimney, door, or window? Knowing how to deliver is half the battle!* 🏠

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_gift_requests_by_delivery_pref')(preference STRING)
RETURNS TABLE(delivery_preference STRING, request_count BIGINT, avg_urgency DOUBLE)
COMMENT 'Returns count of requests and average urgency for a delivery preference (e.g., chimney, door).'
RETURN (
  SELECT 
    delivery_preference,
    COUNT(*) as request_count,
    AVG(urgency_level) as avg_urgency
  FROM IDENTIFIER('{catalog_name}.{schema_name}.gift_requests')
  WHERE delivery_preference = preference
  GROUP BY delivery_preference
);
""")

### 🎁 Popular Gifts
*What's trending this year? (Besides peace on earth).* 📈

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_popular_gifts_by_category')(category STRING)
RETURNS TABLE(item_name STRING, request_count BIGINT)
COMMENT 'Returns the top 5 most requested gifts in a specific category.'
RETURN (
  SELECT 
    gift_item.item_name,
    COUNT(*) as request_count
  FROM IDENTIFIER('{catalog_name}.{schema_name}.gift_requests')
  LATERAL VIEW explode(from_json(extracted_gifts_json, 'ARRAY<STRUCT<item_name: STRING, category: STRING, confidence_score: DOUBLE>>')) AS gift_item
  WHERE primary_gift_category = category
  GROUP BY 1
  ORDER BY 2 DESC
  LIMIT 5
);
""")

### 🚨 High Urgency Requests
*Prioritizing the most critical wishes.* 🆘

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_high_urgency_requests')(min_urgency INT)
RETURNS TABLE(request_id STRING, child_id STRING, urgency_level INT, country STRING)
COMMENT 'Returns a list of gift requests with urgency level greater than or equal to the specified value.'
RETURN (
  SELECT 
    request_id,
    child_id,
    urgency_level,
    country
  FROM IDENTIFIER('{catalog_name}.{schema_name}.gift_requests')
  WHERE urgency_level >= min_urgency
);
""")

### 🌍 Requests by Country
*Mapping out global joy distribution.* 🗺️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_gift_requests_by_country')(country_name STRING)
RETURNS TABLE(country STRING, total_requests BIGINT, avg_sentiment DOUBLE)
COMMENT 'Returns total requests and average sentiment score for a specific country.'
RETURN (
  SELECT 
    country,
    COUNT(*) as total_requests,
    AVG(sentiment_score) as avg_sentiment
  FROM IDENTIFIER('{catalog_name}.{schema_name}.gift_requests')
  WHERE country = country_name
  GROUP BY country
);
""")

### 🦌 Reindeer Flight Metrics
*Checking speed and altitude stats for our antlered aviators.* ✈️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_reindeer_flight_metrics')(reindeer_name STRING)
RETURNS TABLE(reindeer_name STRING, avg_speed_mph DOUBLE, avg_altitude_feet DOUBLE, max_heart_rate INT)
COMMENT 'Returns average flight metrics for a specific reindeer.'
RETURN (
  SELECT 
    reindeer_name,
    AVG(speed_mph) as avg_speed_mph,
    AVG(altitude_feet) as avg_altitude_feet,
    MAX(heart_rate) as max_heart_rate
  FROM IDENTIFIER('{catalog_name}.{schema_name}.reindeer_telemetry')
  WHERE reindeer_name = reindeer_name AND flight_status = 'FLYING'
  GROUP BY reindeer_name
);
""")

### 🛸 Live Flight Status
*Who is currently in the air?* 📡

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_flying_reindeer_status')()
RETURNS TABLE(reindeer_name STRING, current_speed DOUBLE, current_altitude DOUBLE)
COMMENT 'Returns a list of reindeer currently in flight status.'
RETURN (
  SELECT 
    reindeer_name,
    speed_mph as current_speed,
    altitude_feet as current_altitude
  FROM IDENTIFIER('{catalog_name}.{schema_name}.reindeer_telemetry')
  WHERE flight_status = 'FLYING'
  ORDER BY timestamp DESC
  LIMIT 9
);
""")

### 🥕 Low Energy Reindeer
*Identifying who needs a carrot break.* 🥕

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_low_energy_reindeer')(threshold DOUBLE)
RETURNS TABLE(reindeer_name STRING, energy_efficiency DOUBLE, timestamp TIMESTAMP)
COMMENT 'Returns reindeer with energy efficiency below a certain threshold.'
RETURN (
  SELECT 
    reindeer_name,
    energy_efficiency,
    timestamp
  FROM IDENTIFIER('{catalog_name}.{schema_name}.reindeer_telemetry')
  WHERE energy_efficiency < threshold
);
""")

### 🐾 Hoof Pressure Analysis
*Ensuring soft landings on every roof.* 🏠

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_average_hoof_pressure')()
RETURNS TABLE(avg_pressure_fl DOUBLE, avg_pressure_fr DOUBLE, avg_pressure_rl DOUBLE, avg_pressure_rr DOUBLE)
COMMENT 'Returns average hoof pressure for all reindeer across all four limbs.'
RETURN (
  SELECT 
    AVG(hoof_pressure_fl) as avg_pressure_fl,
    AVG(hoof_pressure_fr) as avg_pressure_fr,
    AVG(hoof_pressure_rl) as avg_pressure_rl,
    AVG(hoof_pressure_rr) as avg_pressure_rr
  FROM IDENTIFIER('{catalog_name}.{schema_name}.reindeer_telemetry')
);
""")

### 🚚 Route Metrics
*Optimizing the path from A to B (or North Pole to Everywhere).* 🛣️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_delivery_route_metrics')(start_city STRING, end_city STRING)
RETURNS TABLE(start_city STRING, end_city STRING, avg_distance_km DOUBLE, avg_delivery_window DOUBLE)
COMMENT 'Returns average distance and delivery window for a specific route segment.'
RETURN (
  SELECT 
    start_city,
    end_city,
    AVG(distance_km) as avg_distance_km,
    AVG(delivery_window_minutes) as avg_delivery_window
  FROM IDENTIFIER('{catalog_name}.{schema_name}.delivery_optimization')
  WHERE start_city = start_city AND end_city = end_city
  GROUP BY start_city, end_city
);
""")

### ❄️ Weather Impact
*How much does a blizzard slow us down?* 🌨️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_weather_impact_on_delivery')(weather_condition STRING)
RETURNS TABLE(weather_condition STRING, avg_delay_minutes DOUBLE, count_routes BIGINT)
COMMENT 'Returns average delay minutes caused by a specific weather condition.'
RETURN (
  SELECT 
    weather_condition,
    AVG(weather_delay_minutes) as avg_delay_minutes,
    COUNT(*) as count_routes
  FROM IDENTIFIER('{catalog_name}.{schema_name}.delivery_optimization')
  WHERE weather_condition = weather_condition
  GROUP BY weather_condition
);
""")

### 🧱 Chimney Accessibility
*Calculating the "Santa-fit" factor for each city.* 🎅

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_chimney_accessibility')(city_name STRING)
RETURNS TABLE(city_name STRING, total_households BIGINT, chimneys_available BIGINT, accessibility_rate DOUBLE)
COMMENT 'Returns chimney availability statistics for a given city.'
RETURN (
  SELECT 
    start_city as city_name,
    SUM(households_count) as total_households,
    SUM(chimneys_available) as chimneys_available,
    SUM(chimneys_available) / SUM(households_count) as accessibility_rate
  FROM IDENTIFIER('{catalog_name}.{schema_name}.delivery_optimization')
  WHERE start_city = city_name
  GROUP BY start_city
);
""")

### 📜 Naughty or Nice Score
*The most important metric of all.* ⚖️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_child_naughty_nice_score')(child_id STRING)
RETURNS TABLE(child_id STRING, current_score DOUBLE, gift_tier STRING)
COMMENT 'Returns the current running score and gift tier recommendation for a child.'
RETURN (
  SELECT 
    child_id,
    running_score as current_score,
    gift_tier_recommendation as gift_tier
  FROM IDENTIFIER('{catalog_name}.{schema_name}.behavioral_analytics')
  WHERE child_id = child_id
  ORDER BY timestamp DESC
  LIMIT 1
);
""")

### 🎭 Behavior Event Analysis
*Tracking kindness, sharing, and... tantrums.* 📊

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_behavior_events_by_type')(event_type STRING)
RETURNS TABLE(event_type STRING, event_count BIGINT, avg_impact DOUBLE)
COMMENT 'Returns statistics for a specific type of behavioral event.'
RETURN (
  SELECT 
    event_type,
    COUNT(*) as event_count,
    AVG(impact_score) as avg_impact
  FROM IDENTIFIER('{catalog_name}.{schema_name}.behavioral_analytics')
  WHERE event_type = event_type
  GROUP BY event_type
);
""")

### 🌟 Top Nice Children
*The gold standard of good behavior.* 🏆

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_top_nice_children')()
RETURNS TABLE(child_id STRING, running_score DOUBLE, country STRING)
COMMENT 'Returns the top N children with the highest nice scores.'
RETURN (
  SELECT 
    child_id,
    MAX(running_score) as running_score,
    first(location_country) as country
  FROM IDENTIFIER('{catalog_name}.{schema_name}.behavioral_analytics')
  GROUP BY child_id
  ORDER BY running_score DESC
  LIMIT 25
);
""")

### ⚫ Coal Warning List
*It's not too late to turn it around!* ⚠️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_coal_warning_list')()
RETURNS TABLE(child_id STRING, running_score DOUBLE, warning_date TIMESTAMP)
COMMENT 'Returns a list of children currently in the COAL_WARNING tier.'
RETURN (
  SELECT 
    child_id,
    running_score,
    timestamp as warning_date
  FROM IDENTIFIER('{catalog_name}.{schema_name}.behavioral_analytics')
  WHERE gift_tier_recommendation = 'COAL_WARNING'
  ORDER BY timestamp DESC
);
""")